In [2]:
F = [0, 1]
for i in range(2, 10*n):
    F.append(F[-2] + F[-1])
for i in range(n, 10*n):
    if
    
return(F[-1])

1

In [1]:
import pandas as pd
import numpy as np
import gc
import pathlib
import time

In [2]:
%run "S:\YandexDisk\code\tasks\Tools.ipynb"
%run "S:\YandexDisk\code\tasks\EDA.ipynb"
%run "S:\YandexDisk\code\tasks\Preparation.ipynb"
%run "S:\YandexDisk\code\tasks\Cleaning.ipynb"
%run "S:\YandexDisk\code\tasks\Models.ipynb"
%run "S:\YandexDisk\code\tasks\Selection.ipynb"

In [3]:
# df = pd.read_csv('train.csv', usecols=['user_id', 'content_type_id'],
#                 )

# user_counts = df.user_id.value_counts(sort=False).to_dict()
# l_indexes = df[df.content_type_id == 1].index.to_list()

# del df
# gc.collect()

In [4]:
dtypes = {'timestamp': 'int64',
                            'user_id': 'int32',
                            'content_id': 'int16',
#                             'task_container_id': 'int16',
                            'content_type_id': 'int8',
                            'answered_correctly': 'int8',
                            'prior_question_elapsed_time': 'float64',
                            'prior_question_had_explanation': 'float32'
                        }

In [5]:
stats_train = pd.read_csv('train.csv',
                          usecols=[
                              'timestamp', 'user_id', 'content_id', 'content_type_id', 'answered_correctly',
                              'prior_question_elapsed_time',
                              'prior_question_had_explanation'
                          ], dtype=dtypes, nrows=100000
                         )
stats_train = stats_train[stats_train.content_type_id == 0]
stats_train.sort_values(['timestamp', 'content_id'], inplace=True)
stats_train.drop(columns=['timestamp', 'content_type_id'], inplace=True)

In [6]:
raw_questions = pd.read_csv('questions.csv')

In [7]:
stats_train = stats_train.merge(raw_questions[['question_id', 'bundle_id']],
                                how='left', left_on='content_id', right_on='question_id').drop(columns='question_id')

In [8]:
stats_train[['current_question_elapsed_time',
       'current_question_had_explanation']] = stats_train[['user_id', 'bundle_id', 'prior_question_elapsed_time',
           'prior_question_had_explanation'
       ]].groupby(['user_id', 'bundle_id'], sort=False)[['user_id',
           'prior_question_elapsed_time', 'prior_question_had_explanation'
       ]].tail(1).groupby('user_id', sort=False).shift(-1)

stats_train.drop(columns=['prior_question_elapsed_time', 'prior_question_had_explanation', 'bundle_id'], inplace=True)

In [9]:
stats_train[['current_question_elapsed_time',
       'current_question_had_explanation']] = stats_train[[
           'user_id', 'current_question_elapsed_time',
           'current_question_had_explanation'
       ]].groupby('user_id', sort=False).bfill()

stats_train.drop(columns='user_id', inplace=True)

In [10]:
stats_train = stats_train.groupby('content_id').mean()

stats_train.columns = ['q_mean_answer', 'q_mean_time', 'q_mean_explain']

reduce_mem_usage(stats_train)

gc.collect()

100%|██████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 1503.69it/s]


Mem. usage decreased to  0.17 Mb (42.9% reduction)


23

In [11]:
raw_train = pd.read_csv('train.csv',
                        nrows=100000,
                        dtype=dtypes,
                        usecols=dtypes.keys())
train = raw_train[raw_train.content_type_id == 0]

In [12]:
questions = pd.get_dummies(raw_questions.fillna('-1'), columns=['part'],
                           dtype='int8')

for k in range(188):
    questions['tag_'+str(k)] = np.int8(0)
    
for i in questions.index:
    for tag in questions.loc[i, 'tags'].split():
        if tag != '-1':
            questions.loc[i, 'tag_'+tag] = np.int8(1)
questions.drop(columns=['tags', 'bundle_id', 'correct_answer'], inplace=True)

In [13]:
part_columns = [column for column in questions.columns if column.startswith('part_')]
tag_q_columns = [column for column in questions.columns if column.startswith('tag_')]

In [14]:
reduce_mem_usage(questions)

100%|██████████████████████████████████████████████████████████████████████████████| 196/196 [00:00<00:00, 2806.07it/s]


Mem. usage decreased to  2.54 Mb (3.0% reduction)


In [15]:
train = train.merge(stats_train, how='left', left_on='content_id', right_index=True).merge(questions, how='left', left_on='content_id', right_on='question_id').drop(columns=['timestamp', 'user_id', 'content_type_id'])

In [16]:
train.fillna(0, inplace=True)

In [17]:
y = train.answered_correctly
X = train.drop(columns='answered_correctly')
gc.collect()
reduce_mem_usage(X)

100%|██████████████████████████████████████████████████████████████████████████████| 202/202 [00:00<00:00, 1965.71it/s]


Mem. usage decreased to 20.69 Mb (2.6% reduction)


In [18]:
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

In [ ]:
'lgbm': LGBMClassifier(bagging_fraction=0.7533221689286496, bagging_frec=7,
                boosting_type='dart', cross_entropy_lambda=0.426968949676875,
                feature_fraction=0.8822665705146429,
                learning_rate=0.05710508089711924, min_data_in_leaf=29,
                min_sum_hessian_in_leaf=0.04520091887593929, n_estimators=10000,
                n_jobs=0, num_leaves=5, random_state=13, rank_xendcg=0,
                tree_learner='voting')

In [19]:
MM = Lin_Clf({'xgb': XGBClassifier(base_score=None, booster='dart',
               colsample_bylevel=0.9856746831496574,
               colsample_bynode=0.9725356540133903,
               colsample_bytree=0.9699735716014128, eval_metric='auc',
               gamma=0.6874288311979624, gpu_id=None, importance_type='gain',
               interaction_constraints=None, learning_rate=0.0997832449331652,
               max_delta_step=None, max_depth=2,
               min_child_weight=1.6072393941586012, missing=None,
               monotone_constraints=None, n_estimators=1000, n_jobs=-1,
               num_parallel_tree=None, random_state=13,
               reg_alpha=7.001722155030981, reg_lambda=2.919373935691308,
               scale_pos_weight=0.45830734040341026,
               subsample=0.7704873391200728, tree_method='hist',
               use_label_encoder=False, validate_parameters=None, verbosity=1),
 'lgbm': LGBMClassifier(bagging_fraction=0.7533221689286496, bagging_frec=7,
                boosting_type='dart', cross_entropy_lambda=0.426968949676875,
                feature_fraction=0.8822665705146429,
                learning_rate=0.05710508089711924, min_data_in_leaf=29,
                min_sum_hessian_in_leaf=0.04520091887593929, n_estimators=10000,
                n_jobs=0, num_leaves=5, random_state=13, rank_xendcg=0,
                tree_learner='voting'),
 'cat': CatBoostClassifier(grow_policy='Lossguide',
                           loss_function='CrossEntropy',
                           n_estimators=1000,
                          random_state=13,
                          early_stopping_rounds=10).set_params(**{'learning_rate': 0.003884865909496643,
  'num_leaves': 45,
  'l2_leaf_reg': 5.770755376622499,
  'min_data_in_leaf': 7,
  'colsample_bylevel': 0.44895662503292777,
  'od_pval': 0.05386832639403449,
  'langevin': True})})

In [20]:
MM.val(X, y)

  0%|                                                                                            | 0/3 [00:00<?, ?it/s]

xgb validated in 2178.944325685501


Validate xgb:  33%|██████████████████                                    | 1/3 [36:18<1:12:37, 2178.95s/it, score=0.79]

lgbm validated in 194.59932780265808


Validate lgbm:  67%|████████████████████████████████████                  | 2/3 [39:33<26:23, 1583.65s/it, score=0.792]

cat validated in 540.3093078136444


Validate cat: 100%|████████████████████████████████████████████████████████| 3/3 [48:33<00:00, 971.29s/it, score=0.791]


lgbm    0.7916
cat     0.7912
xgb     0.7896
Name: val_scores, dtype: float64

In [20]:
mm = BO_Clf(X, y)

In [21]:
mm.fit('xgb', 100)

[WARNING 12-14 23:58:38] ax.service.ax_client: Random seed set to 13. Note that this setting only affects the Sobol quasi-random generator and BoTorch-powered Bayesian optimization models. For the latter models, setting random seed to the same number for two optimizations will make the generated trials similar, but not exactly the same, and over time the trials will diverge more.
[INFO 12-14 23:58:38] ax.modelbridge.dispatch_utils: Using Bayesian Optimization generation strategy: GenerationStrategy(name='Sobol+GPEI', steps=[Sobol for 10 trials, GPEI for subsequent trials]). Iterations after 10 will take longer to generate due to  model-fitting.
Optimize xgb: 100%|████████████████████████████████████████████████████| 100/100 [1:04:29<00:00, 38.70s/it, best=0.792]


In [22]:
mm.fit('lgbm', 100)

[WARNING 12-15 01:03:08] ax.service.ax_client: Random seed set to 13. Note that this setting only affects the Sobol quasi-random generator and BoTorch-powered Bayesian optimization models. For the latter models, setting random seed to the same number for two optimizations will make the generated trials similar, but not exactly the same, and over time the trials will diverge more.
[INFO 12-15 01:03:08] ax.modelbridge.dispatch_utils: Using Bayesian Optimization generation strategy: GenerationStrategy(name='Sobol+GPEI', steps=[Sobol for 10 trials, GPEI for subsequent trials]). Iterations after 10 will take longer to generate due to  model-fitting.
Optimize lgbm: 100%|███████████████████████████████████████████████████| 100/100 [2:24:31<00:00, 86.72s/it, best=0.792]


In [23]:
mm.fit('cat', 100)

[WARNING 12-15 03:27:39] ax.service.ax_client: Random seed set to 13. Note that this setting only affects the Sobol quasi-random generator and BoTorch-powered Bayesian optimization models. For the latter models, setting random seed to the same number for two optimizations will make the generated trials similar, but not exactly the same, and over time the trials will diverge more.
[INFO 12-15 03:27:39] ax.modelbridge.dispatch_utils: Using Bayesian Optimization generation strategy: GenerationStrategy(name='Sobol+GPEI', steps=[Sobol for 7 trials, GPEI for subsequent trials]). Iterations after 7 will take longer to generate due to  model-fitting.
Optimize cat: 100%|███████████████████████████████████████████████████| 100/100 [5:47:27<00:00, 208.48s/it, best=0.791]


In [24]:
mm.fit('lgbm', 100)

Optimize lgbm:  58%|████████████████████████████▉                     | 58/100 [1:26:11<1:02:24, 89.16s/it, best=0.792]


KeyboardInterrupt: 

In [25]:
mm.fit('cat', 100)

Optimize cat:   0%|                                                                            | 0/100 [04:22<?, ?it/s]


KeyboardInterrupt: 

In [26]:
sgd      0.7913
lsvm     0.7892
log      0.7891
svm      0.7626
nusvm    0.6850

ridge         0.7891
et            0.7729
rf            0.7724
passagr       0.7448
perceptron    0.5812

SyntaxError: invalid syntax (<ipython-input-26-c6d10df8f23d>, line 1)

In [29]:
{'xgb': XGBClassifier(base_score=None, booster='dart',
               colsample_bylevel=0.9856746831496574,
               colsample_bynode=0.9725356540133903,
               colsample_bytree=0.9699735716014128, eval_metric='auc',
               gamma=0.6874288311979624, gpu_id=None, importance_type='gain',
               interaction_constraints=None, learning_rate=0.0997832449331652,
               max_delta_step=None, max_depth=2,
               min_child_weight=1.6072393941586012, missing=nan,
               monotone_constraints=None, n_estimators=100, n_jobs=-1,
               num_parallel_tree=None, random_state=13,
               reg_alpha=7.001722155030981, reg_lambda=2.919373935691308,
               scale_pos_weight=0.45830734040341026,
               subsample=0.7704873391200728, tree_method='hist',
               use_label_encoder=False, validate_parameters=None, verbosity=1),
 'lgbm': LGBMClassifier(bagging_fraction=0.7533221689286496, bagging_frec=7,
                boosting_type='dart', cross_entropy_lambda=0.426968949676875,
                feature_fraction=0.8822665705146429,
                learning_rate=0.05710508089711924, min_data_in_leaf=29,
                min_sum_hessian_in_leaf=0.04520091887593929, n_estimators=1000,
                n_jobs=0, num_leaves=5, random_state=13, rank_xendcg=0,
                tree_learner='voting'),
 'cat': CatBoostClassifier(grow_policy='Lossguide',
                           loss_function='CrossEntropy',
                           n_estimators=100,
                          random_state=13,
                          early_stopping_rounds=10).set_paramsa({'learning_rate': 0.003884865909496643,
  'num_leaves': 45,
  'l2_leaf_reg': 5.770755376622499,
  'min_data_in_leaf': 7,
  'colsample_bylevel': 0.44895662503292777,
  'od_pval': 0.05386832639403449,
  'langevin': True})}

{'xgb': XGBClassifier(base_score=None, booster='dart',
               colsample_bylevel=0.9856746831496574,
               colsample_bynode=0.9725356540133903,
               colsample_bytree=0.9699735716014128, eval_metric='auc',
               gamma=0.6874288311979624, gpu_id=None, importance_type='gain',
               interaction_constraints=None, learning_rate=0.0997832449331652,
               max_delta_step=None, max_depth=2,
               min_child_weight=1.6072393941586012, missing=nan,
               monotone_constraints=None, n_estimators=100, n_jobs=-1,
               num_parallel_tree=None, random_state=13,
               reg_alpha=7.001722155030981, reg_lambda=2.919373935691308,
               scale_pos_weight=0.45830734040341026,
               subsample=0.7704873391200728, tree_method='hist',
               use_label_encoder=False, validate_parameters=None, verbosity=1),
 'lgbm': LGBMClassifier(bagging_fraction=0.7533221689286496, bagging_frec=7,
                boosti

In [27]:
{'log': LogisticRegression(C=1.0738061095180789, class_weight='balanced',
                    l1_ratio=0.18128745307453936, n_jobs=-1,
                    penalty='elasticnet', random_state=13, solver='saga'),
 'lsvm': LinearSVC(C=0.3481775022639455, class_weight='balanced',
           intercept_scaling=9.020087640834973, max_iter=100, random_state=13),
 'sgd': SGDClassifier(alpha=0.06074084641405568, early_stopping=True,
               eta0=1.096972089773044, l1_ratio=0.6736432360485196,
               learning_rate='adaptive', n_iter_no_change=10, n_jobs=-1,
               penalty='elasticnet', random_state=13),
 'svm': SVC(C=2.8982867976098556, class_weight='balanced', coef0=0.04831821843981743,
     degree=4, gamma='auto', kernel='poly', max_iter=1000, random_state=13),
 'nusvm': NuSVC(class_weight='balanced', coef0=0.13960661925375462, degree=10,
       max_iter=1000, nu=0.20077348545193674, random_state=13),
 'rf': RandomForestClassifier(class_weight='balanced_subsample', max_depth=23,
                        max_features='sqrt', min_samples_leaf=19,
                        min_samples_split=10, n_jobs=-1, random_state=13),
 'et': ExtraTreesClassifier(bootstrap=True, class_weight='balanced_subsample',
                      max_depth=34, max_features='sqrt', min_samples_leaf=20,
                      min_samples_split=10, n_jobs=-1, random_state=13),
 'perceptron': Perceptron(alpha=0.6632954650755191, early_stopping=True,
            eta0=0.20719259265810253, max_iter=100, n_iter_no_change=10,
            n_jobs=-1, penalty='elasticnet', random_state=13),
 'passagr': PassiveAggressiveClassifier(C=0.14488888735380398, early_stopping=True,
                             max_iter=100, n_iter_no_change=10, n_jobs=-1,
                             random_state=13),
 'ridge': RidgeClassifier(alpha=8.93413134585424, class_weight='balanced', max_iter=100,
                 random_state=13, solver='saga')}

{'log': LogisticRegression(C=1.0738061095180789, class_weight='balanced',
                    l1_ratio=0.18128745307453936, n_jobs=-1,
                    penalty='elasticnet', random_state=13, solver='saga'),
 'lsvm': LinearSVC(C=0.3481775022639455, class_weight='balanced',
           intercept_scaling=9.020087640834973, max_iter=100, random_state=13),
 'sgd': SGDClassifier(alpha=0.06074084641405568, early_stopping=True,
               eta0=1.096972089773044, l1_ratio=0.6736432360485196,
               learning_rate='adaptive', n_iter_no_change=10, n_jobs=-1,
               penalty='elasticnet', random_state=13),
 'svm': SVC(C=2.8982867976098556, class_weight='balanced', coef0=0.04831821843981743,
     degree=4, gamma='auto', kernel='poly', max_iter=1000, random_state=13),
 'nusvm': NuSVC(class_weight='balanced', coef0=0.13960661925375462, degree=10,
       max_iter=1000, nu=0.20077348545193674, random_state=13),
 'rf': RandomForestClassifier(class_weight='balanced_subsample', max_dep

In [28]:
{'log': {'C': 1.0738061095180789, 'l1_ratio': 0.18128745307453936},
 'lsvm': {'C': 0.3481775022639455,
  'intercept_scaling': 9.020087640834973,
  'penalty': 'l2',
  'loss': 'squared_hinge'},
 'sgd': {'alpha': 0.06074084641405568,
  'l1_ratio': 0.6736432360485196,
  'eta0': 1.096972089773044,
  'loss': 'hinge',
  'learning_rate': 'adaptive'},
 'svm': {'C': 2.8982867976098556,
  'degree': 4,
  'coef0': 0.04831821843981743,
  'kernel': 'poly',
  'gamma': 'auto'},
 'nusvm': {'nu': 0.20077348545193674,
  'degree': 10,
  'coef0': 0.13960661925375462,
  'kernel': 'rbf',
  'gamma': 'scale'},
 'rf': {'max_depth': 23,
  'min_samples_split': 10,
  'min_samples_leaf': 19,
  'max_features': 'sqrt'},
 'et': {'max_depth': 34,
  'min_samples_split': 10,
  'min_samples_leaf': 20,
  'max_features': 'sqrt'},
 'perceptron': {'alpha': 0.6632954650755191,
  'eta0': 0.20719259265810253,
  'penalty': 'elasticnet'},
 'passagr': {'C': 0.14488888735380398, 'loss': 'hinge'},
 'ridge': {'alpha': 8.93413134585424}}

{'log': {'C': 1.0738061095180789, 'l1_ratio': 0.18128745307453936},
 'lsvm': {'C': 0.3481775022639455,
  'intercept_scaling': 9.020087640834973,
  'penalty': 'l2',
  'loss': 'squared_hinge'},
 'sgd': {'alpha': 0.06074084641405568,
  'l1_ratio': 0.6736432360485196,
  'eta0': 1.096972089773044,
  'loss': 'hinge',
  'learning_rate': 'adaptive'},
 'svm': {'C': 2.8982867976098556,
  'degree': 4,
  'coef0': 0.04831821843981743,
  'kernel': 'poly',
  'gamma': 'auto'},
 'nusvm': {'nu': 0.20077348545193674,
  'degree': 10,
  'coef0': 0.13960661925375462,
  'kernel': 'rbf',
  'gamma': 'scale'},
 'rf': {'max_depth': 23,
  'min_samples_split': 10,
  'min_samples_leaf': 19,
  'max_features': 'sqrt'},
 'et': {'max_depth': 34,
  'min_samples_split': 10,
  'min_samples_leaf': 20,
  'max_features': 'sqrt'},
 'perceptron': {'alpha': 0.6632954650755191,
  'eta0': 0.20719259265810253,
  'penalty': 'elasticnet'},
 'passagr': {'C': 0.14488888735380398, 'loss': 'hinge'},
 'ridge': {'alpha': 8.93413134585424}